In [3]:
import geopandas as gpd
  # Adicionando openpyxl para suporte a Excel
import pandas as pd

# 1. Carregar os arquivos GPKG
gdf_ago = gpd.read_file("C:/Users/Dellg/tracbell/desmatamento_ago22.gpkg")
gdf_set = gpd.read_file("C:/Users/Dellg/tracbell/desmatamento_set22.gpkg")

# 2. Unir os datasets
gdf_unido = gpd.GeoDataFrame(pd.concat([gdf_ago, gdf_set], ignore_index=True))

# 3. Transformar para a projeção EPSG:31983 (SIRGAS 2000 / UTM zone 23S)
gdf_unido = gdf_unido.to_crs(epsg=31983)

# Visualizar as 5 primeiras linhas após transformação
print("\nPrimeiras 5 linhas após transformação de CRS:")
print(gdf_unido.head())


Primeiras 5 linhas após transformação de CRS:
   ogc_fid    id  Instância Data Inicial  Data Final   Cena Score  \
0      1.0  6726       9696   26/07/2022  05/08/2022  22KFC   0.9   
1      2.0  6727       9700   21/07/2022  26/07/2022  22KFD   0.9   
2      3.0  6728       9701   21/07/2022  26/07/2022  22KFD   0.9   
3      4.0  6729       9702   21/07/2022  05/08/2022  22KFD  0.93   
4      5.0  6730       9703   21/07/2022  05/08/2022  22KFD  0.91   

  Data Processamento    Bioma              NuFis Cia PM  \
0   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
1   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
2   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
3   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
4   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   

                                            geometry  
0  POLYGON ((65610.256 7779397.013, 65659.326 777...  
1  POLYGON ((31696.271 7842130.438, 31989.678 784...  
2  POLYGON ((50597

In [7]:
import folium
from folium.plugins import HeatMap

# Centralizar o mapa em Minas Gerais
m = folium.Map(location=[-18.5122, -44.5550], zoom_start=6)

# Converter geometrias para pontos centrais
gdf_unido['coords'] = gdf_unido['geometry'].apply(lambda x: [x.centroid.y, x.centroid.x])

# Criar mapa de calor
HeatMap(gdf_unido['coords'].tolist(), 
        radius=15, 
        blur=10, 
        min_opacity=0.5).add_to(m)

folium.LayerControl().add_to(m)
m.save('heatmap_desmatamento.html')
m

In [11]:
import plotly.express as px

# Verifique qual coluna representa o tipo de vegetação/bioma
# Substitua 'bioma' pelo nome correto da sua coluna (pode ser 'vegetacao_tipo', 'classe', etc.)
coluna_vegetacao = 'Bioma'  # ← ALTERE PARA O NOME CORRETO DA SUA COLUNA

# Criar o gráfico de pizza corrigido
fig = px.pie(gdf_unido,
             names=coluna_vegetacao,  # Coluna com categorias de vegetação
             values='area_ha',        # Coluna com valores numéricos
             title='Distribuição do Desmatamento por Tipo de Vegetação',
             hole=0.4,
             color_discrete_sequence=px.colors.sequential.RdBu)  # Esquema de cores adequado

# Melhorar a formatação
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    marker=dict(line=dict(color='white', width=1)),
    pull=[0.05 if i == gdf_unido[coluna_vegetacao].value_counts().idxmax() 
          else 0 for i in gdf_unido[coluna_vegetacao].unique()]  # Destacar a maior fatia
)

# Ajustar layout
fig.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode='hide',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

fig.show()

ValueError: Value of 'values' is not the name of a column in 'data_frame'. Expected one of ['ogc_fid', 'id', 'Instância', 'Data Inicial', 'Data Final', 'Cena', 'Score', 'Data Processamento', 'Bioma', 'NuFis', 'Cia PM', 'geometry', 'coords'] but received: area_ha